In [14]:
!pip install pinecone

In [75]:
import requests
import pinecone
import json
from datetime import datetime


In [143]:
!pip install openai

   ---------------------------------------- 0.0/386.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/386.9 kB ? eta -:--:--
   ---------- ----------------------------- 102.4/386.9 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------  378.9/386.9 kB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 386.9/386.9 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/199.0 kB ? eta -:--:--
   ---------------------------------------- 199.0/199.0 kB ? eta 0:00:00


In [87]:
import os
from pinecone import Pinecone, ServerlessSpec


pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY", "1b4e6eb1-0270-4ded-b66f-ca77c8d5b980"),
    environment="us-east-1" 
)


index_name = "traveljourney"
embedding_dimension = 3072  


if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=embedding_dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",  
            region="us-west-1"  
        )
    )
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists.")



Index 'traveljourney' already exists.


In [95]:


with open("C:\\Users\\saikr\\Downloads\\Journey_Details.json") as f:
    data = json.load(f)


In [159]:
def create_vector_representation(segment):
    
    return [float(len(segment["flight_number"]))] * 3072 

In [137]:
def flatten_metadata(segment):
    return {
        "flight_number": segment["flight_number"],
        "departure_airport": segment["departure"]["airport"],
        "departure_iata": segment["departure"]["iata"],
        "departure_date": segment["departure"]["date"],
        "arrival_airport": segment["arrival"]["airport"],
        "arrival_iata": segment["arrival"]["iata"],
        "arrival_date": segment["arrival"]["date"]
    }

In [139]:
for flight in data["user"]["flights"]:
    for segment in flight["segments"]:
        
        vector = create_vector_representation(segment)
        
        
        metadata = flatten_metadata(segment)
        
        
        index.upsert(
            vectors=[
                {
                    "id": segment["flight_number"],   
                    "values": vector,                 
                    "metadata": metadata              
                }
            ]
        )

In [157]:
import openai
import pinecone


def create_vector_representation_from_text(query):
    
    embedding = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=query
    )
    return embedding['data'][0]['embedding']  

def retrieve_info(query):
    
    query_vector = create_vector_representation_from_text(query)

    
    search_results = index.query(query_vector, top_k=3, include_metadata=True)
    return search_results

def generate_response(query):
    
    search_results = retrieve_info(query)

    
    prompt = (
        f"User asked: '{query}'. Based on the relevant information: {search_results}, "
        "generate a helpful and context-aware response."
    )
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        max_tokens=150
    )
    return response.choices[0].text


In [155]:
def handle_flight_time_query(flight_number):
    
    try:
        
        response = index.query(vector=None, filter={"flight_number": flight_number}, top_k=1, include_metadata=True)
        
        if response and response['matches']:
            segment = response['matches'][0]['metadata']
            departure_time = segment.get("departure_date", "N/A")
            arrival_time = segment.get("arrival_date", "N/A")
            return f"Your flight {flight_number} departs at {departure_time} and arrives at {arrival_time}."
        else:
            return f"No details found for flight {flight_number}."
    except Exception as e:
        return f"An error occurred: {e}"

def handle_layover_query():
    try:
        
        layover_duration = data["user"]["flights"][0].get("layover_duration", "N/A")
        return f"Your layover is {layover_duration}."
    except KeyError:
        return "Layover duration information is not available."


print(handle_flight_time_query("ET846"))
print(handle_layover_query())


An error occurred: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 29 Oct 2024 12:54:11 GMT', 'Content-Type': 'application/json', 'Content-Length': '76', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1', 'x-pinecone-request-id': '6965481041917244887', 'x-envoy-upstream-service-time': '1', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Either 'vector' or 'ID' must be provided","details":[]}

Your layover is 55m.


In [153]:
def handle_question(query):
    if "flight time" in query:
       
        flight_number = "ET846" 
        return handle_flight_time_query(flight_number)
    
    elif "layover" in query:
        
        return handle_layover_query()
    
    else:
        
        return generate_response(query)


In [151]:
handle_question("can you tell me about the layover time for my journey?")

'Your layover is 55m.'